# Modelisation supervisee

## A. Pre-requis et Nettoyage

In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import mlflow
import mlflow.sklearn
from collections import Counter
import joblib

from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import logging
logging.disable(logging.WARNING)

In [2]:
df = pd.read_csv("dataset/dataset-clean-v2.csv")

## B. Features Engineering

In [3]:
def convert_columns(text):
    """
    Convertit une chaîne de caractères en liste de mots en remplaçant certains caractères spéciaux et en séparant les éléments.

    Args:
        text (str): La chaîne de caractères à traiter. Celle-ci peut contenir des crochets, des virgules et des guillemets.

    Returns:
        list: Une liste de mots extraits de la chaîne, avec les caractères spéciaux remplacés par des espaces.
    """
    return text.replace("[", " ").replace("]", " ").replace(",", " ").replace("'", " ").split()

In [4]:
# Appliquer la fonction 'convert_columns' aux variables 'preproc_tags', 'preproc_body', 'preproc_title'

df['preproc_title'] = df['preproc_title'].apply(convert_columns)
df['preproc_body'] = df['preproc_body'].apply(convert_columns)
df['preproc_tags'] = df['preproc_tags'].apply(convert_columns)

In [5]:
# Fonction pour le pre-processing du texte

# Charger le modèle de langue
nlp = spacy.load("en_core_web_sm")

# Liste des mots spécifiques à conserver (comme "C++", "C#", etc.)
whitelist = ["c#", "c++"]

# Fonction pour le traitement du texte
def preprocess_text(text):
    # Enlever les balises HTML
    #soup = BeautifulSoup(text, "lxml").get_text()
    doc = nlp(text)  # Traiter le texte avec spaCy
    
    # Expression régulière pour vérifier les caractères anglais uniquement (ASCII)
    def is_english(token):
        return re.match(r'^[a-zA-Z0-9+.#]+$', token)

    tokens = [
        # Utiliser le lemme sauf si le mot est dans la whitelist
        token.lemma_.lower() if token.lemma_.lower() not in whitelist else token.text.lower()
        for token in doc 
        if not token.is_stop                         # Ne pas inclure les stopwords
        and not token.is_punct                       # Ne pas inclure la ponctuation
        and not token.like_num                       # Ne pas inclure les chiffres
        and len(token.lemma_) >= 3                    # Exclure les tokens trop courts
        and (is_english(token.text) or token.text.lower() in whitelist)  # Garder les mots anglais ou ceux de la whitelist
    ]
    print(tokens)
    return tokens

In [6]:
# Fusionner les listes 'preproc_body' et 'preproc_title'

df['preproc_corpus'] = df.apply(lambda row: row['preproc_body'] + row['preproc_title'], axis=1)

In [7]:
# Creation de la variable 'corpus' qui regroupe les colonnes 'preproc_title' et 'preproc_body'

df['corpus'] = df['preproc_title'].apply(lambda x: ' '.join(x)) + ' ' + df['preproc_body'].apply(lambda x: ' '.join(x))

In [8]:
# Compter la fréquence des tags dans toute la colonne 'preproc_tags'
top_100_tags = [tag for tag, count in Counter(tag for tags in df['preproc_tags'] for tag in tags).most_common(100)]
print(top_100_tags)

['c#', 'java', 'c++', 'javascript', 'php', 'asp.net', 'iphone', 'jquery', 'python', 'sql', 'html', 'c', 'sql-server', 'objective-c', 'mysql', 'database', 'windows', 'wpf', 'linux', 'ajax', 'performance', 'xml', 'css', 'ruby-on-rails', 'ruby', 'asp.net-mvc', 'flash', 'cocoa-touch', 'multithreading', 'visual-studio', 'cocoa', 'image', 'django', 'macos', 'apache-flex', 'web-services', 'string', 'security', 'arrays', 'visual-studio-2008', 'actionscript-3', 'user-interface', 'vb.net', 'algorithm', 't-sql', 'hibernate', 'sql-server-2005', 'forms', 'eclipse', 'oracle', 'debugging', 'winforms', 'internet-explorer', 'file', 'winapi', 'unit-testing', 'http', 'parsing', 'json', 'xcode', 'events', 'spring', 'class', 'xaml', 'apache', 'linq', 'android', 'delphi', 'unix', 'authentication', 'data-binding', 'firefox', 'silverlight', 'optimization', 'ios', 'wcf', 'jakarta-ee', 'regex', 'oop', 'memory', 'gcc', 'orm', 'perl', 'network-programming', 'iis', 'shell', 'validation', 'swing', 'visual-c++', 'me

In [9]:
df[['preproc_title', 'preproc_tags', 'preproc_body']]

,preproc_title,preproc_tags,preproc_body
0,"[gettimeofday, guarantee, microsecond, resolut...","[linux, winapi, visual-c++, unix, timer]","[port, game, originally, write, win32, api, li..."
1,"[decode, sql, cast]","[c#, sql, vb.net, ascii, hex]","[recently, site, deluge, resurgence, asprox, b..."
2,"[display, flash, content, winforms, application]","[c#, winforms, flash, adobe, macromedia]","[good, way, display, flash, content, winforms,..."
3,"[sql, text, indexing, return, result, word, co...","[sql, sql-server, sql-server-2005, indexing, f...","[instance, query, like, follow, sql, server, s..."
4,"[unit, test, flex, application, ide, build, sc...","[apache-flex, eclipse, unit-testing, build-aut...","[currently, work, application, frontend, write..."
...,...,...,...
49995,"[possible, prevent, iphone, ipad, orientation,...","[iphone, ipad, webkit, rotation, orientation]","[see, similar, question, issue, relate, native..."
49996,"[log, fatal, parse, error, php5]","[php, fatal-error, error-logging, parse-error,...","[write, error, log, service, integrate, websit..."
49997,"[address, use, java]","[java, web-applications, jboss, ip-address, st...","[time, restart, application, build, java, stru..."
49998,"[selenium, testing]","[perl, selenium, selenium-rc, selenium-ide, gu...","[want, bunch, dom, object, xpath, loop, check,..."


In [10]:
# Filtrer les tags dans 'preproc_tags' en ne gardant que ceux présents dans le top 100
df['filtered_tags'] = df['preproc_tags'].apply(lambda tags: [tag for tag in tags if tag in top_100_tags])

In [11]:
# Supprimer les individus qui n'ont plus de tags
df = df[df['filtered_tags'].apply(len) > 0]

## C. Modelisation

In [12]:
# Preparation de MLFLow
mlflow.set_experiment("StackOverflow Tags Prediction 2")

<Experiment: artifact_location='file:///Users/vdumont/pCloud%20Drive/Formation%20OpenClassroom/Projets/Projet%205/mlruns/269961575685329476', creation_time=1727802178758, experiment_id='269961575685329476', last_update_time=1727802178758, lifecycle_stage='active', name='StackOverflow Tags Prediction 2', tags={}>

### 0. Preparation du jeu d'entrainement et de test

In [13]:
# Répartition des données
X_train, X_test, y_train, y_test = train_test_split(df['corpus'], df['filtered_tags'], test_size=0.2, random_state=42)

### 1. Regression Logistique avec Bag-of-Word

In [14]:
with mlflow.start_run(run_name="RegLog - BoW"):
    # Log des paramètres du modèle dans MLFlow
    mlflow.log_param("max_iter", 1000)
    # Bag of Words sur le jeu d'entraînement X_train
    vectorizer_bow = CountVectorizer()
    X_train_bow = vectorizer_bow.fit_transform(X_train)

    # Préparer les étiquettes (tags) avec MultiLabelBinarizer sur le jeu d'entraînement y_train
    mlb = MultiLabelBinarizer()
    y_mlb = mlb.fit_transform(y_train)

    # Modelisation RegLog avec BoW
    model_bow = OneVsRestClassifier(LogisticRegression(max_iter=1000))
    model_bow.fit(X_train_bow, y_mlb)

    # Predire les tags sur le jeu de test X_test
    y_pred_bow = model_bow.predict(vectorizer_bow.transform(X_test))

    # Préparer les étiquettes (tags) du jeu de test y_test
    y_test_mlb = mlb.transform(y_test)

    # Evaluation des performances du modèle avec Bag of Words

    accuracy = accuracy_score(y_test_mlb, y_pred_bow)
    precision = precision_score(y_test_mlb, y_pred_bow, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_bow, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_bow, average='weighted')

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_bow, "model_reglog_bow")

    print(f"""Regression Logistique avec Bag of Words :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

### 2. Regression Logistique avec TF-IDF

In [ ]:
with mlflow.start_run(run_name="RegLog - TF-IDF"):
    # Log des paramètres du modèle dans MLFlow
    mlflow.log_param("max_iter", 1000)
    # TF-IDF sur le jeu d'entrainement X_train
    vectorizer_tfidf = TfidfVectorizer()
    X_tfidf = vectorizer_tfidf.fit_transform(X_train)

    # Préparer les étiquettes (tags) avec MultiLabelBinarizer sur le jeu d'entraînement y_train
    mlb = MultiLabelBinarizer()
    y_mlb = mlb.fit_transform(y_train)

    # Modèle Regression Logistique avec TF-IDF
    model_tfidf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
    model_tfidf.fit(X_tfidf, y_mlb)

    # Predire les tags sur le jeu de test X_test
    y_pred_tfidf = model_tfidf.predict(vectorizer_tfidf.transform(X_test))

    # Préparer les étiquettes (tags) du jeu de test y_test
    y_test_mlb = mlb.transform(y_test)

    accuracy = accuracy_score(y_test_mlb, y_pred_tfidf)
    precision= precision_score(y_test_mlb, y_pred_tfidf, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_tfidf, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_tfidf, average='weighted')

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_tfidf, "model_reglog_tfidf")

    print(f"""Regression Logistique avec TF-IDF :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

In [16]:
def predict_top_n_tags(input_sentence, vectorizer, model, mlb, n=5):
    """
    Prédit les 'n' tags les plus probables pour une phrase d'entrée en utilisant le modèle et le vecteur BoW.

    Args:
        input_sentence (str): La phrase d'entrée à partir de laquelle prédire les tags.
        vectorizer (CountVectorizer): Le vecteur BoW déjà ajusté sur le jeu d'entraînement.
        model (OneVsRestClassifier): Le modèle de régression logistique ajusté.
        mlb (MultiLabelBinarizer): L'encodeur MultiLabelBinarizer ajusté sur les étiquettes de tags.
        n (int): Le nombre de tags à prédire (par défaut 5).

    Returns:
        list: Une liste des 'n' tags les plus probables pour l'entrée.
    """

    # Transformer la phrase en BoW
    X_input_bow = vectorizer.transform([input_sentence])

    # Obtenir les probabilités des prédictions
    y_pred_prob = model.predict_proba(X_input_bow)

    # Sélectionner les indices des 'n' tags les plus probables
    top_n_indices = np.argsort(y_pred_prob[0])[-n:]

    # Convertir ces indices en tags
    top_n_tags = mlb.classes_[top_n_indices]

    # Retourner les 'n' tags prédits
    return list(top_n_tags)

In [17]:
input_sentence = "How to compile a C programm using gcc in Ubuntu?"

In [ ]:
# Exemple d'utilisation sur le Bag of Words (BoW):
top_n = 5  # Par exemple, choisir 3 tags
top_n_predicted_tags = predict_top_n_tags(input_sentence, vectorizer_bow, model_bow, mlb, n=top_n)
print(f"Les {top_n} tags prédits pour la phrase '{input_sentence}' sont : {top_n_predicted_tags}")

In [ ]:
# Exemple d'utilisation sur le TF-IDF :
top_n = 5  # Par exemple, choisir 3 tags
top_n_predicted_tags = predict_top_n_tags(input_sentence, vectorizer_tfidf, model_tfidf, mlb, n=top_n)
print(f"Les {top_n} tags prédits pour la phrase '{input_sentence}' sont : {top_n_predicted_tags}")

### 3. Regression Logistique avec Word2Vec

#### Embedding

In [20]:
# Tokenisation du corpus pour Word2Vec (supposant que chaque document est une liste de tokens)
sentences_train = [doc.split() for doc in X_train]
sentences_test = [doc.split() for doc in X_test]

# Entraînement du modèle Word2Vec
w2v_model = Word2Vec(sentences=sentences_train, vector_size=100, window=5, min_count=2, workers=4)

# Fonction pour obtenir la moyenne des embeddings d'un document
def get_avg_word2vec(tokens_list, vector, size):
    # On garde les vecteurs uniquement des tokens présents dans le vocabulaire de Word2Vec
    valid_words = [word for word in tokens_list if word in vector.wv.index_to_key]
    if len(valid_words) == 0:  # Si aucun mot n'est reconnu, retourner un vecteur nul
        return np.zeros(size)
    return np.mean([vector.wv[word] for word in valid_words], axis=0)

# Calculer les embeddings moyens pour chaque document
X_train_w2v = np.array([get_avg_word2vec(doc.split(), w2v_model, 100) for doc in X_train])
X_test_w2v = np.array([get_avg_word2vec(doc.split(), w2v_model, 100) for doc in X_test])

# Préparer les étiquettes (tags) avec MultiLabelBinarizer sur le jeu d'entraînement y_train
mlb = MultiLabelBinarizer()
y_mlb = mlb.fit_transform(y_train)

#### Modelisation

In [ ]:
with mlflow.start_run(run_name="RegLog - Word2Vec"):
    # Log des paramètres du modèle dans MLFlow
    mlflow.log_param("max_iter", 1000)
    # Modélisation RegLog avec Word2Vec
    model_w2v = OneVsRestClassifier(LogisticRegression(max_iter=1000))
    model_w2v.fit(X_train_w2v, y_mlb)

    # Prédire les tags sur le jeu de test X_test
    y_pred_w2v = model_w2v.predict(X_test_w2v)

    # Préparer les étiquettes (tags) du jeu de test y_test
    y_test_mlb = mlb.transform(y_test)

    accuracy = accuracy_score(y_test_mlb, y_pred_w2v)
    precision = precision_score(y_test_mlb, y_pred_w2v, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_w2v, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_w2v, average='weighted')

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_w2v, "model_reglog_word2vec")

    print(f"""Regression Logistique avec Word2Vec :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

### 4. DecisionTreeClassifier avec Word2Vec

In [ ]:
with mlflow.start_run(run_name="DecisionTreeClassifier - Word2Vec"):
    # Modélisation DecisionTreeClassifier avec Word2Vec
    model_w2v_decisiontree = OneVsRestClassifier(DecisionTreeClassifier())
    model_w2v_decisiontree.fit(X_train_w2v, y_mlb)

    # Prédire les tags sur le jeu de test X_test
    y_pred_w2v = model_w2v_decisiontree.predict(X_test_w2v)

    # Préparer les étiquettes (tags) du jeu de test y_test
    y_test_mlb = mlb.transform(y_test)

    accuracy = accuracy_score(y_test_mlb, y_pred_w2v)
    precision = precision_score(y_test_mlb, y_pred_w2v, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_w2v, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_w2v, average='weighted')

    metrics = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1
        }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_bow, "model_decisiontree_word2vec")

    print(f"""Arbre de décision avec Word2Vec :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

In [38]:
# Fonction pour prédire les X tags les plus probables d'une nouvelle phrase
def predict_top_tags(phrase, w2v_model, classifier, mlb, vector_size=100, top_n=5):
    # Prétraitement de la phrase (tokenisation)
    tokens = preprocess_text(phrase)
    
    # Calculer la moyenne des embeddings de la phrase
    phrase_embedding = get_avg_word2vec(tokens, w2v_model, vector_size)
    
    # Le classificateur a besoin d'un tableau 2D, donc on reshape le vecteur
    phrase_embedding = phrase_embedding.reshape(1, -1)
    
    # Obtenir les probabilités des prédictions
    y_pred_proba = classifier.predict_proba(phrase_embedding)
    
    # Sélectionner les indices des X plus grandes probabilités
    top_n_indices = np.argsort(y_pred_proba[0])[-top_n:][::-1]
    
    # Récupérer les tags correspondants à ces indices
    top_n_tags = mlb.classes_[top_n_indices]
    
    return top_n_tags

In [ ]:
# Prediction de tags avec Regression Logistique et Word2Vec
phrase_input = "How to compile a C programm using gcc in Ubuntu?"
top_n_tags = predict_top_tags(phrase_input, w2v_model, model_w2v, mlb, top_n=5)

print(f"RegLog W2V - Top 3 tags prédits pour la phrase '{phrase_input}': {top_n_tags}")

In [ ]:
# Prediction de tags avec DecisitonTreeClassifier et Word2Vec
phrase_input = "How to compile a C programm using gcc in Ubuntu?"
top_n_tags = predict_top_tags(phrase_input, w2v_model, model_w2v_decisiontree, mlb, top_n=5)

print(f"DecisionTree W2V - Top 3 tags prédits pour la phrase '{phrase_input}': {top_n_tags}")

### 5. BERT

In [ ]:
'''import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from transformers import AutoTokenizer, TFAutoModel

model_type = "bert-base-uncased"
model = TFAutoModel.from_pretrained(model_type)
tokenizer = AutoTokenizer.from_pretrained(model_type)

def get_embeddings_in_batches(texts, batch_size=16):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='tf')
        batch_embeddings = model(tokens['input_ids'])['pooler_output']
        embeddings_list.append(batch_embeddings)
    return tf.concat(embeddings_list, axis=0)

X_train_list = X_train.tolist()
X_test_list = X_test.tolist()

X_train_emdebbing = get_embeddings_in_batches(X_train_list, batch_size=32)
X_test_emdebbing = get_embeddings_in_batches(X_test_list, batch_size=32)

# Modélisation Regression Logistique avec BERT
model_bert_reglog = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model_bert_reglog.fit(X_train_emdebbing, y_mlb)

# Prédire les tags sur le jeu de test X_test
y_pred_w2v = model_bert_reglog.predict(X_test_emdebbing)

# Préparer les étiquettes (tags) du jeu de test y_test
y_test_mlb = mlb.transform(y_test)

accuracy = accuracy_score(y_test_mlb, y_pred_w2v)
precision = precision_score(y_test_mlb, y_pred_w2v, average='weighted')
recall = recall_score(y_test_mlb, y_pred_w2v, average='weighted')
f1 = f1_score(y_test_mlb, y_pred_w2v, average='weighted')

print(f"""Regression Logistique avec BERT :
Score Accuracy : {accuracy:.4f}
Score Precision : {precision:.4f}
Score Recall : {recall:.4f}
Score F1 : {f1:.4f}""")

import pickles
with open('models/model-bert-reglog.pkl', 'wb') as f:
    pickle.dump(model_bert_reglog, f)'''

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from transformers import AutoTokenizer, TFAutoModel

model_type = "bert-base-uncased"
model = TFAutoModel.from_pretrained(model_type)
tokenizer = AutoTokenizer.from_pretrained(model_type)

def get_embeddings_in_batches(texts, batch_size=16):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='tf')
        batch_embeddings = model(tokens['input_ids'])['pooler_output']
        embeddings_list.append(batch_embeddings)
    return tf.concat(embeddings_list, axis=0)

with mlflow.start_run(run_name="RegLog - BERT"):
    # Chargement du modele BERT RegLog
    mlflow.log_param("max_iter", 1000)
    model_bert_reglog, X_test_emdebbing, y_pred_bert = joblib.load('models/model_bert_relog.joblib')

    y_pred_bert = model_bert_reglog.predict(X_test_emdebbing)
    y_test_mlb = mlb.transform(y_test)

    accuracy = accuracy_score(y_test_mlb, y_pred_bert)
    precision = precision_score(y_test_mlb, y_pred_bert, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_bert, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_bert, average='weighted')

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_bert_reglog, "model_bert_reglog")

    print(f"""Regression Logistique avec BERT :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

### 6. USE

In [ ]:
'''use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Fonction pour générer les embeddings USE
def get_use_embeddings(texts):
    embeddings = use_model(texts)
    return np.array(embeddings)

X_train_use = get_use_embeddings(X_train.tolist())
X_test_use = get_use_embeddings(X_test.tolist())

model_use_reglog = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model_use_reglog.fit(X_train_use, y_mlb)

y_pred_use = model_use_reglog.predict(X_test_use)

# Préparer les étiquettes (tags) du jeu de test y_test
y_test_mlb = mlb.transform(y_test)

accuracy = accuracy_score(y_test_mlb, y_pred_use)
precision = precision_score(y_test_mlb, y_pred_use, average='weighted')
recall = recall_score(y_test_mlb, y_pred_use, average='weighted')
f1 = f1_score(y_test_mlb, y_pred_use, average='weighted')

print(f"""Regression Logistique avec Bag of Words :
Score Accuracy : {accuracy:.4f}
Score Precision : {precision:.4f}
Score Recall : {recall:.4f}
Score F1 : {f1:.4f}""")

import pickle

with open('models/model-use-reglog.pkl', 'wb') as f:
    pickle.dump(model_use_reglog, f)'''

In [22]:

with mlflow.start_run(run_name="RegLog - BERT"):
    mlflow.log_param("max_iter", 1000)
    # Chargement du modele USE RegLog
    model_use_reglog, X_test_use, y_pred_use = joblib.load('models/model_use_relog.joblib')

    y_pred_use = model_bert_reglog.predict(X_test_use)
    y_test_mlb = mlb.transform(y_test)

    accuracy = accuracy_score(y_test_mlb, y_pred_use)
    precision = precision_score(y_test_mlb, y_pred_use, average='weighted')
    recall = recall_score(y_test_mlb, y_pred_use, average='weighted')
    f1 = f1_score(y_test_mlb, y_pred_use, average='weighted')

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model_bert_reglog, "model_use_reglog")

    print(f"""Regression Logistique avec USE :
    Score Accuracy : {accuracy:.4f}
    Score Precision : {precision:.4f}
    Score Recall : {recall:.4f}
    Score F1 : {f1:.4f}""")

In [ ]:
!mlflow ui